# Test Case LVV-T13517
The objective of this test case is to verify the MTAlignment communication interface with SAL by testing the commands, events and telemetry defined by the latest version of the XML. This test case will exercise the functionality of the alignment system on the TTS and 3rd level of the Summit and meets the following criteria:

    Only requires the MTAlignment to be operational
    Does not require any communication with other subsystems
    Does require the use of SAL and the EFD
    


In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

## Setup

In [6]:
import asyncio
import logging
import os
import yaml

import astropy.units as u
import numpy as np
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst.ts import MTAlignment
ip = "192.168.0.2"
port = 50000
from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger
from lsst_efd_client import EfdClient
from lsst.ts import salobj

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [7]:
os.environ["LSST_DDS_HISTORYSYNC"] = "1"

Setting up logger

In [8]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

Getting unique index for Script

In [9]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

Make sure DDS Daemon is running and startup Domain

In [6]:
domain = salobj.Domain()

Instantiate script for logging into EFD and start script task

In [7]:
test_message = "LVV-T13517 Interface to SAL"
script = salobj.Controller("Script", index=index)
await script.start_task

EFD setup

In [31]:
client = vandv.efd.create_efd_client()

MTAlignment Initialization

In [10]:
await MTAlignment.start_task

[None, None, None]

In [130]:
await MTAlignment.standby()

Helper functions: Event callback and telemetry check

In [33]:
def evt_time_callback(evt):
    """Print event and time when it is received."""
    time_evt = datetime.utcfromtimestamp(evt.private_sndStamp-37).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    evt_name = str(evt.__class__).split('_')[2]
    print(f'\n {evt_name} logevent at {time_evt} is \n \t{evt}')

In [34]:
async def check_telemetry(topic):
    tel = await client.select_top_n(topic,'*',num=1)
    time_tel = datetime.utcfromtimestamp(tel['private_efdStamp'][0]).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    print(f'Last {topic} telemetry was at {time_tel} UTC \n \tvs time now is {Time.now()}')

Publish in the EFD that test is starting

In [14]:
script.log.info(f'START- {test_message} -- at {Time(datetime.now())}')

--- 
## Step 1:
LVV-T13517 (1.0) MTAlignment Corrections 1